In [1]:
#The outputs displayed in this file are for the facebook network dataset

#The inputs and outputs for self-testing have been stored in Input_a.txt and Output_a.txt respectively
#The inputs and outputs for toy example have been stored in Input_b.txt and Output_b.txt respectively
#The inputs and outputs for facebook network dataset have been stored in Input_c.txt and Output_c.txt respectively

In [2]:
import pandas as pd
import numpy as np
import sys

In [3]:
graph = input("Is your graph directed or undirected. Enter 'D' for Directed Graph and 'UD' for Undirected Graph: \n")

Is your graph directed or undirected. Enter 'D' for Directed Graph and 'UD' for Undirected Graph: 
 UD


In [4]:
#Storing the user input as a text file 
if graph == 'D':
    def get_input():
        print("Enter the text you want to store in the file. Press 0 to finish.")
        lines = []
        while True:
            line = input()
            if line == '0':
                break
            lines.append(line)
        return '\n'.join(lines)

    def save_to_file(content, file_path):
        with open(file_path, 'w') as file:
            file.write(content)

    user_input = get_input()

    file_path = input("Enter the path where you want to save the input file (including the filename and extension): ")

    try:
        save_to_file(user_input, file_path)
        print(f"Content saved to '{file_path}' successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

    #Creating a dataframe of the output and input nodes
    custom_headers = ['Outgoing Link', 'Incoming Link']
    df = pd.read_csv(file_path, header=None, names=custom_headers, sep=' ')

    print(df)


elif graph == 'UD':
    file_path = input("Enter the path where you want to save the input file (including the filename and extension): ")

    custom_headers = ['Outgoing Link', 'Incoming Link']
    df = pd.read_csv(file_path, header=None, names=custom_headers, sep=' ')

    #Since this is for undirected graph, we have to consider two links for every relationship
    new_rows = df.copy()
    new_rows['Incoming Link'] = df['Outgoing Link']
    new_rows['Outgoing Link'] = df['Incoming Link']

    df = pd.concat([df, new_rows], ignore_index=True)
    
    print(df)

Enter the path where you want to save the input file (including the filename and extension):  /Users/harshshah/Downloads/Input_a.txt


  Outgoing Link Incoming Link
0             A             Y
1             A             M
2             Y             Y
3             Y             A
4             M             A
5             Y             A
6             M             A
7             Y             Y
8             A             Y
9             A             M


In [5]:
#Creating a dataframe with outgoing nodes as columns and incoming nodes as rows
unique_values_column1 = df['Outgoing Link'].unique().tolist()

new_df = pd.DataFrame(0, index=unique_values_column1, columns=unique_values_column1, dtype=float)

print(new_df)

     A    Y    M
A  0.0  0.0  0.0
Y  0.0  0.0  0.0
M  0.0  0.0  0.0


In [6]:
#Counting the out-degree for each node
counts_column1 = df['Outgoing Link'].value_counts()

print(counts_column1)

Outgoing Link
A    4
Y    4
M    2
Name: count, dtype: int64


In [7]:
#Dividing 1 by the out-degree count for each node and assigning that value
counts_column1 = df['Outgoing Link'].value_counts()

counts_dict_column1 = counts_column1.to_dict()

for key in counts_dict_column1:
    counts_dict_column1[key] = 1/counts_dict_column1[key]

In [8]:
#Creating a dataframe by inserting all the values for the incoming and outgoing nodes.
for i in range(len(df)):
    a = df['Outgoing Link'][i]
    b = df['Incoming Link'][i]
    new_df[a][b] = counts_dict_column1[a]

print(new_df)

      A     Y    M
A  0.00  0.25  0.5
Y  0.25  0.25  0.0
M  0.25  0.00  0.0


In [9]:
M = new_df.to_numpy()

print(M)

[[0.   0.25 0.5 ]
 [0.25 0.25 0.  ]
 [0.25 0.   0.  ]]


In [10]:
#Each node has an equal probability of being the start node. So assigning equal probability to each node.
r_t1 = np.full((len(new_df), 1), 1/len(new_df))

print(r_t1)

[[0.33333333]
 [0.33333333]
 [0.33333333]]


In [11]:
#Actual calculation of the PageRank for each node by using the Power Iteration method
def distance(Matrix, r1):
    e = sys.float_info.epsilon
    z = 1
    while(z >= e):
        r2 = np.matmul(Matrix, r1)
        z = np.linalg.norm(r2 - r1)
        
        if z < e:
            return r2
            break
        else: r1 = r2

In [12]:
final = distance(M, r_t1)

print(final)

[[8.88174501e-17]
 [8.88179916e-17]
 [4.44091632e-17]]


In [13]:
final_df = pd.DataFrame(data = final, index = unique_values_column1, columns = ['Rank'])

print(final_df)

           Rank
A  8.881745e-17
Y  8.881799e-17
M  4.440916e-17


In [14]:
#Final sorted dataframe in descending order of ranks 
final_sorted_df = final_df.sort_values(by='Rank', ascending=False)

print(final_sorted_df)

           Rank
Y  8.881799e-17
A  8.881745e-17
M  4.440916e-17


In [15]:
#The sum of all ranks should be equal to 1. Verifying that.
#Rounding the final answer since sometimes for very large graphs, sum might be equal to 0.9999... which is basically 1.
#The sum is 0.9999... sometimes because of decimal multiplication

column_sum = final_sorted_df['Rank'].sum()

print(f"The sum of Ranks of all the nodes is {column_sum} which when rounded off is {round(column_sum)}.")

The sum of Ranks of all the nodes is 2.2204460492503128e-16 which when rounded off is 0.


In [16]:
#Storing the output to a text file
file_path_op = input("Enter the path where you want to save the output file (including the filename and extension): ") 

try:
    final_sorted_df.to_csv(file_path_op, index = True, sep = ' ', header = False, float_format = '%.6f')
    print(f"Content saved to '{file_path_op}' successfully.")
except Exception as e:
    print(f"An error occurred: {e}")

Enter the path where you want to save the output file (including the filename and extension):  /Users/harshshah/Downloads/Output_c.txt


Content saved to '/Users/harshshah/Downloads/Output_c.txt' successfully.
